In [ ]:
# Fuente: 
# https://aboveintelligent.com/image-segmentation-with-neural-net-d5094d571b1e
# https://github.com/mzaradzki/neuralnets/tree/master/vgg_segmentation_keras

In [ ]:
!apt install python-pydot python-pydot-ng graphviz 
!pip install pydot

In [ ]:
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Deconvolution2D, Cropping2D
from keras.layers import Input, Add, Dropout, Permute, add
from keras.utils import plot_model
from scipy.io import loadmat
from scipy.misc import imread
import matplotlib.pyplot as plt
from PIL import Image
import copy
from scipy.misc import bytescale
from scipy.io import loadmat

In [ ]:
# Function to create to a series of CONV layers followed by Max pooling layer
def convblock(cdim, nb, nfilt=3):
	L = []

	for k in range(1, nfilt + 1):
		convname = 'conv' + str(nb) + '_' + str(k)
		L.append(Convolution2D(cdim, kernel_size=(3, 3), padding='same', activation='relu', name=convname))

	L.append(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

	return L

In [ ]:
#Helper function to create Sequential part of the Architecture
def fcn32_blank(image_size=512):
	
  mdl = Sequential()

  # First layer is a dummy-permutation = Identity to specify input shape
  mdl.add(Permute((1, 2, 3), input_shape=(image_size, image_size, 3)))  # WARNING : axis 0 is the sample dim

  for l in convblock(64, 1, nfilt=2):
    mdl.add(l)

  for l in convblock(128, 2, nfilt=2):
    mdl.add(l)

  for l in convblock(256, 3, nfilt=3):
    mdl.add(l)

  for l in convblock(512, 4, nfilt=3):
    mdl.add(l)

  for l in convblock(512, 5, nfilt=3):
    mdl.add(l)

  mdl.add(Convolution2D(4096, kernel_size=(7, 7), padding='same', activation='relu', name='fc6'))  # WARNING border
  mdl.add(Convolution2D(4096, kernel_size=(1, 1), padding='same', activation='relu', name='fc7'))  # WARNING border
  
  # WARNING : model decapitation i.e. remove the classifier step of VGG16 (usually named fc8)

  mdl.add(Convolution2D(21, kernel_size=(1, 1), padding='same', activation='relu', name='score_fr'))

  convsize = mdl.layers[-1].output_shape[2]
  deconv_output_size = (convsize - 1) * 2 + 4  # INFO: =34 when images are 512x512
  mdl.add(Deconvolution2D(21, kernel_size=(4, 4), strides=(2, 2), padding='valid', activation=None, name='score2'))

  extra_margin = deconv_output_size - convsize * 2  # INFO: =2 when images are 512x512
  assert (extra_margin > 0)
  assert (extra_margin % 2 == 0)
  # INFO : cropping as deconv gained pixels
  c = ((0, extra_margin), (0, extra_margin))
  mdl.add(Cropping2D(cropping=((extra_margin/2, extra_margin/2), (extra_margin/2, extra_margin/2))) ) # INFO : cropping as deconv gained pixels
  
  return mdl

In [ ]:
def fcn_32s_to_16s(fcn32model=None):
  if fcn32model is None:
    fcn32model = fcn32_blank()

  fcn32shape = fcn32model.layers[-1].output_shape
  assert (len(fcn32shape) == 4)
  assert (fcn32shape[0] is None)  # batch axis
  assert (fcn32shape[3] == 21)  # number of filters
  assert (fcn32shape[1] == fcn32shape[2])  # must be square

  fcn32size = fcn32shape[1]  # INFO: =32 when images are 512x512

  if fcn32size != 32:
    print('WARNING : handling of image size different from 512x512 has not been tested')

  sp4 = Convolution2D(21, kernel_size=(1, 1), padding='same', activation=None, name='score_pool4')

  # INFO : to replicate MatConvNet.DAGN.Sum layer see documentation at :
  # https://keras.io/getting-started/sequential-model-guide/
  summed = add(inputs=[sp4(fcn32model.layers[14].output), fcn32model.layers[-1].output])

  # INFO :
  # final 16x16 upsampling of "summed" using deconv layer upsample_new (32, 32, 21, 21)
  # deconv setting is valid if (528-32)/16 + 1 = deconv_input_dim (= fcn32size)
  deconv_output_size = (fcn32size - 1) * 16 + 32  # INFO: =528 when images are 512x512
  upnew = Deconvolution2D(21, kernel_size=(32, 32),
              padding='valid',  # WARNING : valid, same or full ?
              strides=(16, 16),
              activation=None,
              name='upsample_new')

  extra_margin = deconv_output_size - fcn32size * 16  # INFO: =16 when images are 512x512
  assert (extra_margin > 0)
  assert (extra_margin % 2 == 0)
  # INFO : cropping as deconv gained pixels
  crop_margin = Cropping2D(cropping=((extra_margin/2, extra_margin/2), (extra_margin/2, extra_margin/2))) # INFO : cropping as deconv gained pixels

  return Model(fcn32model.input, crop_margin(upnew(summed)))

In [ ]:
def prediction(kmodel, crpimg, transform=False):
	# INFO : crpimg should be a cropped image of the right dimension

	imarr = np.array(crpimg).astype(np.float32)

	if transform:
		imarr[:, :, 0] -= 129.1863
		imarr[:, :, 1] -= 104.7624
		imarr[:, :, 2] -= 93.5940
		#
		# WARNING : in this script (https://github.com/rcmalli/keras-vggface) colours are switched
		aux = copy.copy(imarr)
		imarr[:, :, 0] = aux[:, :, 2]
		imarr[:, :, 2] = aux[:, :, 0]

	imarr = np.expand_dims(imarr, axis=0)

	return kmodel.predict(imarr)

In [ ]:
fcn32model = fcn32_blank()
print(fcn32model.summary())

In [ ]:
fcn16model = fcn_32s_to_16s(fcn32model)
print(fcn16model.summary())

In [ ]:
plot_model(fcn16model,'FCN-16_withshape.png',show_shapes=True)
plt.figure(figsize=(150,40))
plt.imshow(imread('FCN-16_withshape.png'))

In [ ]:
!wget http://www.vlfeat.org/matconvnet/models/pascal-fcn16s-dag.mat

In [ ]:
data = loadmat('pascal-fcn16s-dag.mat', matlab_compatible=False, struct_as_record=False)
layers = data['layers']
params = data['params']
description = data['meta'][0,0].classes[0,0].description

In [ ]:
print(data.keys())

In [ ]:
print(type(layers))

In [ ]:
print(layers.shape)

In [ ]:
class2index = {}
for i, clname in enumerate(description[0,:]):
    class2index[str(clname[0])] = i
    
print(sorted(class2index.keys()))

In [ ]:
for i in range(0, params.shape[1]-1, 2):
    print(i,
          str(params[0,i].name[0]), params[0,i].value.shape,
          str(params[0,i+1].name[0]), params[0,i+1].value.shape)

In [ ]:
for i in range(layers.shape[1]):
    print(i,
          str(layers[0,i].name[0]), str(layers[0,i].type[0]),
          [str(n[0]) for n in layers[0,i].inputs[0,:]],
          [str(n[0]) for n in layers[0,i].outputs[0,:]])

In [ ]:
def copy_mat_to_keras(kmodel):
    
    kerasnames = [lr.name for lr in kmodel.layers]

    prmt = (0, 1, 2, 3) # WARNING : important setting as 2 of the 4 axis have same size dimension
    
    for i in range(0, params.shape[1]-1, 2):
        matname = '_'.join(params[0,i].name[0].split('_')[0:-1])
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            print('found : ', (str(matname), kindex))
            l_weights = params[0,i].value
            l_bias = params[0,i+1].value
            f_l_weights = l_weights.transpose(prmt)
            if False: # WARNING : this depends on "image_data_format":"channels_last" in keras.json file
                f_l_weights = np.flip(f_l_weights, 0)
                f_l_weights = np.flip(f_l_weights, 1)
            print(f_l_weights.shape, kmodel.layers[kindex].get_weights()[0].shape)
            assert (f_l_weights.shape == kmodel.layers[kindex].get_weights()[0].shape)
            assert (l_bias.shape[1] == 1)
            assert (l_bias[:,0].shape == kmodel.layers[kindex].get_weights()[1].shape)
            assert (len(kmodel.layers[kindex].get_weights()) == 2)
            kmodel.layers[kindex].set_weights([f_l_weights, l_bias[:,0]])
        else:
            print('not found : ', str(matname))

In [ ]:
copy_mat_to_keras(fcn16model)

In [ ]:
!wget http://www.robots.ox.ac.uk/~szheng/crfasrnndemo/static/rgb.jpg
image_size = 512
im = Image.open('rgb.jpg') # http://www.robots.ox.ac.uk/~szheng/crfasrnndemo/static/rgb.jpg
im = im.crop((0,0,319,319)) # WARNING : manual square cropping
im = im.resize((image_size,image_size))
plt.imshow(np.asarray(im))
plt.axis('off')

In [ ]:
crpim = im # WARNING : we deal with cropping in a latter section, this image is already fit
preds = prediction(fcn16model, crpim, transform=True)

In [ ]:
preds.shape

In [ ]:
imclass = np.argmax(preds, axis=3)[0,:,:]


plt.figure(figsize = (15, 7))
plt.subplot(1,3,1)
plt.imshow(crpim)
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(imclass, cmap='jet')
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(crpim)
plt.axis('off')

masked_imclass = np.ma.masked_where(imclass == 0, imclass)
plt.imshow(masked_imclass, alpha=0.5, cmap='jet')
plt.axis('off')

In [ ]:
# List of dominant classes found in the image# List o 
for c in np.unique(imclass):
    print c, str(description[0,c][0])

In [ ]:
bspreds = bytescale(preds, low=0, high=255)

plt.figure(figsize = (15, 7))

plt.subplot(2,3,1)
plt.imshow(np.asarray(crpim))
plt.axis('off')

plt.subplot(2,3,3+1)
plt.imshow(bspreds[0,:,:,class2index['background']], cmap='seismic')
plt.axis('off')

plt.subplot(2,3,3+2)
plt.imshow(bspreds[0,:,:,class2index['person']], cmap='seismic')
plt.axis('off')

plt.subplot(2,3,3+3)
plt.imshow(bspreds[0,:,:,class2index['bicycle']], cmap='seismic')
plt.axis('off')